In [1]:
!pip install meds-tab

In [2]:
#@title Download E-ICU demo
import os
from pathlib import Path
notebook_dir = os.getcwd()

ROOT_DIR=f"{notebook_dir}/work_dir/mimiciv_demo/"
# ROOT_DIR=f"{notebook_dir}/work_dir/eicu_demo/"
Path(ROOT_DIR).mkdir(parents=True, exist_ok=True)

MEDS_DIR = ROOT_DIR + "/meds/"
OUTPUT_TABULARIZATION_DIR= ROOT_DIR + "/tabularized/"
TASK_DIR = MEDS_DIR + "/task_labels"
TASK_NAME="mortality/in_icu/first_24h"
# TASK_NAME="los_in_hospital_first_48h"
OUTPUT_MODEL_DIR= ROOT_DIR + "/output/meds_tab/"

In [3]:
!meds-tab-describe input_dir={MEDS_DIR}/data output_dir={OUTPUT_TABULARIZATION_DIR}

2024-12-14 19:28:24.645 | INFO     | MEDS_tabular_automl.utils:stage_init:446 - Running describe_codes with the following configuration:
input_dir: /Users/sim/Documents/projects/MEDS-DEV/demo/work_dir/mimiciv_demo//meds//data
output_dir: /Users/sim/Documents/projects/MEDS-DEV/demo/work_dir/mimiciv_demo//tabularized/
do_overwrite: false
seed: 1
tqdm: false
worker: 0
loguru_init: false
log_dir: ${output_dir}/.logs/
cache_dir: ${output_dir}/.cache
output_filepath: ${output_dir}/metadata/codes.parquet
name: describe_codes

2024-12-14 19:28:24.646 | DEBUG    | MEDS_tabular_automl.utils:stage_init:467 - Stage config:
input_dir: /Users/sim/Documents/projects/MEDS-DEV/demo/work_dir/mimiciv_demo//meds//data
output_dir: /Users/sim/Documents/projects/MEDS-DEV/demo/work_dir/mimiciv_demo//tabularized/
do_overwrite: false
seed: 1
tqdm: false
worker: 0
loguru_init: false
log_dir: ${output_dir}/.logs/
cache_dir: ${output_dir}/.cache
output_filepath: ${output_dir}/metadata/codes.parquet
name: describe_

In [4]:
# Define the window sizes and aggregations to generate features for
WINDOW_SIZES = "tabularization.window_sizes=[1d,30d,365d]"
AGGREGATIONS = "tabularization.aggs=[static/present,code/count,value/count,value/sum,value/sum_sqd,value/min,value/max]"

In [5]:
# !rm -rf /content/tabularized/tabularize/

In [13]:
!meds-tab-tabularize-static "input_dir=$MEDS_DIR/data" "output_dir=$OUTPUT_TABULARIZATION_DIR" do_overwrite=False "$WINDOW_SIZES" "$AGGREGATIONS"

2024-12-14 19:29:06.167 | INFO     | MEDS_tabular_automl.utils:stage_init:446 - Running tabularize_static with the following configuration:
input_dir: /Users/sim/Documents/projects/MEDS-DEV/demo/work_dir/mimiciv_demo//meds//data
output_dir: /Users/sim/Documents/projects/MEDS-DEV/demo/work_dir/mimiciv_demo//tabularized/
do_overwrite: false
seed: 1
tqdm: false
worker: 0
loguru_init: false
log_dir: ${output_dir}/.logs/
cache_dir: ${output_dir}/.cache
tabularization:
  filtered_code_metadata_fp: ${output_dir}/metadata/codes.parquet
  allowed_codes: null
  min_code_inclusion_count: 10
  min_code_inclusion_frequency: null
  min_correlation: null
  max_by_correlation: null
  max_included_codes: null
  window_sizes:
  - 1d
  - 30d
  - 365d
  aggs:
  - static/present
  - code/count
  - value/count
  - value/sum
  - value/sum_sqd
  - value/min
  - value/max
  _resolved_codes: ${filter_to_codes:${tabularization.filtered_code_metadata_fp},${tabularization.allowed_codes},${tabularization.min_code_i

In [14]:
!meds-tab-tabularize-time-series --multirun "worker=range(0,2)" "hydra/launcher=joblib" "input_dir=$MEDS_DIR/data" "output_dir=$OUTPUT_TABULARIZATION_DIR" do_overwrite=False  "$WINDOW_SIZES" "$AGGREGATIONS"

[2024-12-14 19:29:41,824][HYDRA] Joblib.Parallel(n_jobs=-1,backend=loky,prefer=processes,require=None,verbose=0,timeout=None,pre_dispatch=2*n_jobs,batch_size=auto,temp_folder=None,max_nbytes=None,mmap_mode=r) is launching 2 jobs
[2024-12-14 19:29:41,824][HYDRA] Launching jobs, sweep output dir : /Users/sim/Documents/projects/MEDS-DEV/demo/work_dir/mimiciv_demo/tabularized/.logs
[2024-12-14 19:29:41,824][HYDRA] 	#0 : worker=0 input_dir=/Users/sim/Documents/projects/MEDS-DEV/demo/work_dir/mimiciv_demo//meds//data output_dir=/Users/sim/Documents/projects/MEDS-DEV/demo/work_dir/mimiciv_demo//tabularized/ do_overwrite=False tabularization.window_sizes=[1d,30d,365d] tabularization.aggs=[static/present,code/count,value/count,value/sum,value/sum_sqd,value/min,value/max]
[2024-12-14 19:29:41,824][HYDRA] 	#1 : worker=1 input_dir=/Users/sim/Documents/projects/MEDS-DEV/demo/work_dir/mimiciv_demo//meds//data output_dir=/Users/sim/Documents/projects/MEDS-DEV/demo/work_dir/mimiciv_demo//tabularized/ 

In [15]:
!meds-tab-cache-task "input_dir={MEDS_DIR}/data" "output_dir=$OUTPUT_TABULARIZATION_DIR" "input_label_dir=$TASK_DIR/$TASK_NAME/" "task_name=$TASK_NAME" do_overwrite=False  "$WINDOW_SIZES" "$AGGREGATIONS"

2024-12-14 19:31:32.691 | INFO     | MEDS_tabular_automl.utils:stage_init:446 - Running cache_task with the following configuration:
input_dir: /Users/sim/Documents/projects/MEDS-DEV/demo/work_dir/mimiciv_demo//meds//data
output_dir: /Users/sim/Documents/projects/MEDS-DEV/demo/work_dir/mimiciv_demo//tabularized/
do_overwrite: false
seed: 1
tqdm: false
worker: 0
loguru_init: false
log_dir: ${output_dir}/.logs/
cache_dir: ${output_dir}/.cache
tabularization:
  filtered_code_metadata_fp: ${output_dir}/metadata/codes.parquet
  allowed_codes: null
  min_code_inclusion_count: 10
  min_code_inclusion_frequency: null
  min_correlation: null
  max_by_correlation: null
  max_included_codes: null
  window_sizes:
  - 1d
  - 30d
  - 365d
  aggs:
  - static/present
  - code/count
  - value/count
  - value/sum
  - value/sum_sqd
  - value/min
  - value/max
  _resolved_codes: ${filter_to_codes:${tabularization.filtered_code_metadata_fp},${tabularization.allowed_codes},${tabularization.min_code_inclusio

In [ ]:
!meds-tab-model --multirun "input_dir=$MEDS_DIR/data" "output_dir=$OUTPUT_TABULARIZATION_DIR" "output_model_dir=$OUTPUT_MODEL_DIR/$TASK_NAME/" "task_name=$TASK_NAME" do_overwrite=False "hydra.sweeper.n_trials=10"  "$WINDOW_SIZES" "$AGGREGATIONS" "tabularization.min_code_inclusion_count=10"

[I 2024-12-14 19:36:45,225] A new study created in memory with name: no-name-12520edb-1d03-4285-8296-9a50cda0c57f
[2024-12-14 19:36:45,225][HYDRA] Study name: no-name-12520edb-1d03-4285-8296-9a50cda0c57f
[2024-12-14 19:36:45,225][HYDRA] Storage: None
[2024-12-14 19:36:45,225][HYDRA] Sampler: TPESampler
[2024-12-14 19:36:45,225][HYDRA] Directions: ['maximize']
[2024-12-14 19:36:45,228][HYDRA] Joblib.Parallel(n_jobs=-1,backend=loky,prefer=processes,require=None,verbose=0,timeout=None,pre_dispatch=2*n_jobs,batch_size=auto,temp_folder=None,max_nbytes=None,mmap_mode=r) is launching 2 jobs
[2024-12-14 19:36:45,228][HYDRA] Launching jobs, sweep output dir : /Users/sim/Documents/projects/MEDS-DEV/demo/work_dir/mimiciv_demo/output/meds_tab/mortality/in_icu/first_24h/2024-12-14_19-36-44/hydra
[2024-12-14 19:36:45,228][HYDRA] 	#0 : +model_launcher.model.eta=0.3778393076431474 +model_launcher.model.lambda=0.02025881019876421 +model_launcher.model.alpha=0.0016680628084537149 +model_launcher.model.s